In [25]:
import pandas as pd
import numpy as np
#import plotly.express as px
from scipy.signal import argrelextrema

from datetime import datetime
import warnings
import json
warnings.filterwarnings("ignore")

In [54]:
sample_tsdata = pd.read_csv(r'Time_Series_Data.txt')
#print(sample_tsdata.head())
newcolmname = sample_tsdata.columns.str.replace(' ', '')
sample_tsdata.columns = list(newcolmname)


In [56]:
sample_tsdata.head(n=10)

1. Find positive and negative tend for each report
2. Find records which highest and lowest deviation
3. If the devition with single report or all report
4. Articualte 3 scenario in sentence and fill report name which outlier


,CompTime,RUNDATE,REPORT,DOMAIN,AH_NAME,COMPLETETIME,SLA,Number_of_Records
0,0:45,18/10/21,Sales PO Shipment (01:30),MP,JB.GECSC.CMD.SLACK.SALES.PO.SHPMNT.END.BQ,18/10/21 0:45,1:30,1
1,0:50,18/10/21,Pre-Orders Core (01:30),MP,JB.GECORDER.ORDER.US.SET.DATE.PARAM.SALES.ORDE...,18/10/21 0:50,1:30,1
2,0:58,18/10/21,MP Commissions (02:30),MP,JB.GECFINANCE.GECFINANCE.CMD.SLACK.SOCD.END,18/10/21 0:58,2:30,1
3,0:45,17/10/21,Sales PO Shipment (01:30),MP,JB.GECSC.CMD.SLACK.SALES.PO.SHPMNT.END.BQ,17/10/21 0:45,1:30,1
4,0:47,17/10/21,Pre-Orders Core (01:30),MP,JB.GECORDER.ORDER.US.SET.DATE.PARAM.SALES.ORDE...,17/10/21 0:47,1:30,1
5,1:00,17/10/21,MP Commissions (02:30),MP,JB.GECFINANCE.GECFINANCE.CMD.SLACK.SOCD.END,17/10/21 1:00,2:30,1
6,0:45,16/10/21,Pre-Orders Core (01:30),MP,JB.GECORDER.ORDER.US.SET.DATE.PARAM.SALES.ORDE...,16/10/21 0:45,1:30,1
7,0:45,16/10/21,Sales PO Shipment (01:30),MP,JB.GECSC.CMD.SLACK.SALES.PO.SHPMNT.END.BQ,16/10/21 0:45,1:30,1
8,1:01,16/10/21,MP Commissions (02:30),MP,JB.GECFINANCE.GECFINANCE.CMD.SLACK.SOCD.END,16/10/21 1:01,2:30,1
9,0:39,15/10/21,Pre-Orders Core (01:30),MP,JB.GECORDER.ORDER.US.SET.DATE.PARAM.SALES.ORDE...,15/10/21 0:39,1:30,1


In [ ]:
def create_missed_Sla (x1=sample_tsdata['SLA'],x2=sample_tsdata['CompTime']):
    
    time_diff = x1 - x2
    missed_sla = 'met' if time_diff > 0 else 'missed'
    
    return missed_sla

In [ ]:
sample_tsdata["MISSED_SLA"] = create_missed_Sla (x1=sample_tsdata['SLA'],x2=sample_tsdata['CompTime'])

In [51]:
ts_json=json.dumps(sample_tsdata.to_dict())
ts_param_json = json.dumps({'xaxisname':'CompTime',
            'yaxisname': 'RUNDATE',
            'impcol':['COMPLETETIME'],
            'grp_col':['REPORT'],
            'recency': None,
            'sla_col': 'SLA',
            'date_col': None,
            'specifc_fucn': } )

In [53]:
pd.DataFrame(json.loads(ts_json)).head()

,CompTime,RUNDATE,REPORT,DOMAIN,AH_NAME,COMPLETETIME,SLA,Number_of_Records
0,0:45,18/10/21,Sales PO Shipment (01:30),MP,JB.GECSC.CMD.SLACK.SALES.PO.SHPMNT.END.BQ,18/10/21 0:45,1:30,1
1,0:50,18/10/21,Pre-Orders Core (01:30),MP,JB.GECORDER.ORDER.US.SET.DATE.PARAM.SALES.ORDE...,18/10/21 0:50,1:30,1
2,0:58,18/10/21,MP Commissions (02:30),MP,JB.GECFINANCE.GECFINANCE.CMD.SLACK.SOCD.END,18/10/21 0:58,2:30,1
3,0:45,17/10/21,Sales PO Shipment (01:30),MP,JB.GECSC.CMD.SLACK.SALES.PO.SHPMNT.END.BQ,17/10/21 0:45,1:30,1
4,0:47,17/10/21,Pre-Orders Core (01:30),MP,JB.GECORDER.ORDER.US.SET.DATE.PARAM.SALES.ORDE...,17/10/21 0:47,1:30,1


In [29]:
#sample_tabdata.iloc[:4,:].to_dict()
def convert_datetime(date_str):
    try:
        return datetime.strptime(date_str, "%d/%m/%y %H:%M")
    except:
        return datetime.strptime(date_str, "%d/%m/%Y %H:%M")

def convert_time(time_str):
    a, b = list(map(int, time_str.split(":")))
    return a *60 +  b


# Configurations

In [33]:
# Data receviving

_input = {'tabdata':tab_dict,
 'tabparam':{'target_col':'Profit',
            'impcol':['Sales','Quantity'],
            'grp_cols':['Category'],
             'reporting_cols' : ['Region','State'],
             'recency':['date_col',"no_of_point"],
             'corr_colm_check': None
            },
 'tsdata':ts_json,
 'tsparam':{'xaxisname':'CompTime',
            'yaxisname': 'RUNDATE',
            'impcol':['MISSED_SLA'],
            'grp_col':['REPORT'],
            'recency': None
            }  
  }


tab_knowledge_base = ["Max and Min of {0} is {1}. The {0} is correlated with {2} and could be potential reason for max and min.",
                      "Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.", 
                      "Max {self.target_col.lower()} is {self.max} in category {self.max_grp_col}.",
                      "The {0} is correlated with {2} and could be potential reason."]

ts_knowledge_base = ["In last {self.recency} days, peak is {self.peak} {self.yaxisname} on {self.xaxispoint}).", "The {0} is correlated with {2} and could be potential reason for max and min.",
                      "Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.", 
                      "Max {self.target_col.lower()} is {self.max} in category {self.max_grp_col}.",
                      "The {0} is correlated with {2} and could be potential reason."]


# Data

In [36]:
class Data:
    
    def __init__(self,_input=None):
        self.input = _input
        self.tabdf = None
        self.ts_df = None
        self.tabparam = None
        self.tspram = None
        
    def parseinput(self):
        if self.input['tabdata'] != None:
            #self.tabdf = pd.read_json(self.input['tabdata'])
            self.tabdf = pd.DataFrame(self.input['tabdata'])
            self.tabparam = self.input['tabparam']

        if self.input['tsdata'] != None:    
            self.ts_df = json.loads(self.input['tsdata'])
            self.tsparam = self.input['tsparam']
                

In [10]:
class Nlgtabinsight:
    def __init__ (self, df, tab_param ):
        self.df = df
        self.imp_cols = tab_param['impcol']
        self.target_col = tab_param['target_col']
        self.nm =  tab_param['corr_colm_check']
        self.grp_cols = tab_param['grp_cols']
        self.reporting_cols = tab_param['reporting_cols']
        self.corr_val = None
        self.max = None
        self.max_grp_col = None
        self.xaxismax_name = None
        self.min = None
        self.min_grp_col = None
        self.corr_val = None
        self.corr_col_name = None
        
    
#     def read_file(self):
#         df = pd.read_excel('downloads/Sample - Superstore (1).xls')
#         return df
    
    def get_corr(self):
        corr_df = self.df[self.cols]
#         corr_df = df[self.cols + list(target_col)]* We will not need to pass the target col in the col list 
        corr_coeff = pd.DataFrame(corr_df.corr())
        corr_coeff_val = pd.Series(abs(corr_coeff[self.target_col]))
        corr_coeff_tgt = corr_coeff[corr_coeff_val.between(0.3,0.99)]
        corr_val = corr_coeff_tgt[self.target_col].values
        nm = corr_coeff_tgt.columns[corr_coeff_val.between(0.3,0.99)]
#       print(type(corr_coeff_tgt))
        self.nm = nm[0]
        self.corr_val = corr_val[0]
        
    def getmaxmin(self):
        
        df_temp = self.df.copy()
        bottom_rows = df_temp.loc[df_temp.groupby(self.grp_cols)[self.target_col].idxmin(axis=0)]
        self.min = [str(round(x,1)) for x in list(bottom_rows[self.target_col])]
        self.min_grp_col = [str(x) for x in list(bottom_rows[self.grp_cols[0]])]
       
        top_rows = df_temp.loc[df_temp.groupby(self.grp_cols)[self.target_col].idxmax(axis=0)]
        self.max = [str(round(x,1)) for x in list(top_rows[self.target_col])]
        self.max_grp_col = [str(x) for x in list(top_rows[self.grp_cols[0]])]
    
        
        
    def print_val(self):
        if self.nm != None:
            print(f'Corr between {self.nm} and {self.target_col} is {self.corr_val}')
            
    def get_insight(self):
        global tab_knowledge_base
        
        all_insight = []
        all_insight.append(eval(str("f'")+tab_knowledge_base[1]+"'"))
        all_insight.append(eval(str("f'")+tab_knowledge_base[2]+"'"))
        
        #print(f'Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.')
        self.tab_insight = all_insight
        
            
            

# TS Insight

In [37]:
class Nlgtsinsight:
    
    def __init__ (self, df, ts_param ):
        
        self.df = df'
        self.xaxisname = ts_param['xaxisname']
        self.yaxisname = ts_param['RUNDATE']
        self.imp_cols = ts_param['impcol']
        self.grp_col = ts_param['grp_col']
        self.recency = ts_param['recency'] if ts_param['recency'] != None else None
        self.yaxisname = ts_param['RUNDATE']
        
        {'xaxisname':'RUNDATE',
            'yaxisname': 'CompTime',
            'impcol':['COMPLETETIME'],
            'grp_col':['REPORT'],
            'recency': None,
            'sla_col': 'SLA',
            'date_col': None}  
        
        df["COMPLETE_TIME"] = df[self.date_col].apply(convert_datetime)
        df["COMPTIME_MIN"] = df[self.nm].apply(convert_time)

        self.df = df
        self.imp_cols = ts_param['impcol']
        self.target_col = ts_param['xaxisname']
        self.nm =  ts_param['yaxisname']
        self.grp_cols = ts_param['grp_cols']
        self.reporting_cols = ts_param['reporting_cols']
        self.date_col = ts_param['date_col']
        
    

        
    def drops_and_spikes():
        df = self.df

        
        dfs = [x for _, x in df.groupby(self.grp_cols)]

        for odf in dfs:
            odf = odf.sort_values(by=['COMPLETE_TIME'])
            
            a = argrelextrema(odf['COMPTIME_MIN'].values, np.less_equal, order=3)[0]
            b = argrelextrema(odf['COMPTIME_MIN'].values, np.greater_equal, order=3)[0]
            
            drops.append(odf.iloc[a])
            spikes.append(odf.iloc[b])
        
        return drops, spikes
    
    def achived_sla():
        return len(df[df["SLA_MIN"] < df['COMPTIME_MIN']])

    def missed_sla():
        return len(df[df["SLA_MIN"] < df['COMPTIME_MIN']])

In [9]:
# Tab Insights

In [24]:
if __name__ == '__main__':

    data=Data(_input)
    data.parseinput()
    
    data.
    
    #
    tabinsght = Nlgtabinsight(data.tabdf,data.tabparam)
    temp=tabinsght.getmaxmin()
    
    # TS insight
    

    

NameError: name 'Data' is not defined

In [ ]:
tabinsght.get_insight()

In [ ]:
tabinsght.tab_insight

["Min profit is ['-1665.1', '-1850.9', '-3840.0'] in category ['Furniture', 'Office Supplies', 'Technology'].",
 "Max profit is ['673.9', '3177.5', '3920.0'] in category ['Furniture', 'Office Supplies', 'Technology']."]

In [ ]:
list(temp['Profit'].values)

[-1665.0522, -1850.9464000000007, -3839.9903999999988]

In [ ]:
temp.columns = [x[0]+"_"+x[1] for x in temp.columns]


In [ ]:
temp.columns
rename(columns={'ID':'COUNT','TIME':'MEAN_TIME'})

NameError: name 'rename' is not defined

In [ ]:
temp